<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#作戦メモ" data-toc-modified-id="作戦メモ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>作戦メモ</a></span></li><li><span><a href="#前処理" data-toc-modified-id="前処理-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>前処理</a></span></li><li><span><a href="#優秀なトレーニング期間の探索" data-toc-modified-id="優秀なトレーニング期間の探索-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>優秀なトレーニング期間の探索</a></span></li></ul></div>

# 作戦メモ
トレーニング期間を変えながら、性能が良いモデルを構築できる期間を探し出す。  
そして、それらの期間ごとにモデルを構築し、最後にアンサンブルする。  

このノートブックは作戦探索用である。
提出用ノートブックは別に用意すべし。

In [ ]:
import numpy as np
import lightgbm as lgb
import pandas as pd
from kaggle.competitions import twosigmanews
import matplotlib.pyplot as plt
import random
from datetime import datetime, date
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time
from joblib import Parallel, delayed
from tqdm import tqdm
from dateutil.parser import parse
import gc

import datetime
from dateutil.relativedelta import relativedelta
from IPython import embed

In [ ]:
#データ取得コマンド
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
print('Done!')

In [ ]:
market_train_df = env._var07
news_train_df = env._var10

In [ ]:
config_model = {"n_lag":[3,7,14], # 時系列特徴量計算の周期リスト
                
                # 時系列特徴量計算のリスト
                "return_features" : ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close'],
                
                "validation_term" : ['2017-01-01', '2018-12-31'],
                
                "model_params": [{ # パラメータ１
                                    'task': 'train',
                                    'boosting_type': 'gbdt',
                                    'objective': 'binary',
                                    'learning_rate': 0.05,
                                    'num_leaves': 2400,
                                    'min_data_in_leaf': 150,
                                    'num_iteration': 1000,
                                    'max_bin': 200,
                                    'verbose': 1,
                                    'metric' : 'None'  # カスタム評価関数のみ使う
                                },
                                { # パラメータ２
                                    'task': 'train',
                                    'boosting_type': 'gbdt',
                                    'objective': 'binary',
                                    'learning_rate': 0.048,
                                    'num_leaves': 2300,
                                    'min_data_in_leaf': 150,
                                    'num_iteration': 1000,
                                    'max_bin': 200,
                                    'verbose': 1,
                                    'metric' : 'None'  # カスタム評価関数のみ使う
                                }]
               }


# 前処理

In [ ]:
market_train_df['time'] = market_train_df['time'].dt.date

In [ ]:
def create_lag(df_code,n_lag, return_features):

    #df_codeの資産コードを重複なしでcodeに入れる
    code = df_code['assetCode'].unique()
    
    # データフレーム高速生成のための辞書
    _dict_for_df = {"time": df_code["time"].values,
                    "assetCode": df_code["assetCode"].values}
    shift_size = 1
    for col in return_features:
        for window in n_lag:
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            #移動平均の平均
            lag_mean = rolled.mean()
            #最大値
            lag_max = rolled.max()
            #最小値
            lag_min = rolled.min()
            #標準偏差
            lag_std = rolled.std()
            #colの値_lag_windowの値_meanのカラム名でdf_codeに格納される
            _dict_for_df['%s_lag_%s_mean'%(col,window)] = lag_mean
            _dict_for_df['%s_lag_%s_max'%(col,window)] = lag_max
            _dict_for_df['%s_lag_%s_min'%(col,window)] = lag_min
    df_result = pd.DataFrame.from_dict(_dict_for_df)
    del df_code
    return df_result.fillna(-1)


def generate_lag_features(df,n_lag, return_features):
    all_df = Parallel(n_jobs=-1)(tqdm([delayed(create_lag)(_df, n_lag, return_features) for _code, _df in df.groupby('assetCode')]))
    new_df = pd.concat(all_df)
    del df
    gc.collect()
    return new_df

def data_prep(market_train):
    lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
    market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    market_train = market_train.dropna(axis=0)
    return market_train

def mis_impute(data):
    #カラムを一つずつ引っ張ってくる
    for i in data.columns:
        #型が文字列なら空値をotherに置き換え
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        #型が数字なら空値は平均値に置き換え
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data

def get_feature_cols(df):
    """
    特徴量とするカラムを取得する。
    """
    fcol = [c for c in df.columns if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'time_x','provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'universe','sourceTimestamp']]
    return fcol

In [ ]:
new_df = generate_lag_features(market_train_df,
                               n_lag=config_model['n_lag'], 
                               return_features=config_model['return_features'])
market_train_df = pd.merge(market_train_df,new_df,how='left',on=['time','assetCode'], copy=False)
market_train_df = mis_impute(market_train_df)
market_train_df = data_prep(market_train_df)

#up = market_train_df['returnsOpenNextMktres10'] >= 0

#universe(これが１以外は訓練データとして使えない)の値をuniverse変数に放り込む
#でもなんで値？
#universe = market_train_df['universe'].values


In [ ]:
# メモリ節約
for _ in market_train_df.columns:
    if market_train_df[_].dtype == np.float64:
        market_train_df[_] = market_train_df[_].astype(np.float32)
gc.collect()

# 優秀なトレーニング期間の探索
学習期間をスライドさせながら、バリデーション期間における性能を測定し、記録する。  
そして、優秀な性能を誇る期間をリストアップする。

In [ ]:
# 各学習期間の性能測定結果を記録する。
# 各要素は、以下を記録する。
# "term":学習期間, "sigma_score":スコア, "post_process_param": ポストプロセスパラメータ(シグモイド係数)
performance_records = []

# 特徴量カラム
feature_cols = get_feature_cols(market_train_df)

# インデックス情報
idx_validate = market_train_df[(market_train_df['time'] >= parse(config_model['validation_term'][0]).date())
                               & (market_train_df['time'] <= parse(config_model['validation_term'][1]).date())].index

In [ ]:
# バリデーションデータの用意
X_valid = market_train_df.loc[idx_validate][feature_cols].values
up_valid = market_train_df.loc[idx_validate]['returnsOpenNextMktres10'] > 0
validation_data = lgb.Dataset(X_valid, label=up_valid.astype(int),free_raw_data=False,
                             params={"return":market_train_df.loc[idx_validate]['returnsOpenNextMktres10'].values, 
                                     "time":market_train_df.loc[idx_validate]['time'].values,
                                     "universe": market_train_df.loc[idx_validate]['universe'].values})
gc.collect()

In [ ]:
pos_training_start = parse('2012-01-01')
training_term_range = relativedelta(months=12)  # 学習期間は1年間
training_term_step = relativedelta(weeks=2)
gc.collect()

In [ ]:
def get_training_data(market_train_df, idx_training, feature_cols):
    """
    トレーニングデータを取得する
    """
    return ( market_train_df.loc[idx_training][feature_cols].values,   # X_train
             market_train_df.loc[idx_training]['returnsOpenNextMktres10'] > 0, # up_train
           )

def eval_sigma_score(preds, valid_set):
    """
    予測とバリデーションセットを受け取り、sigma_scoreを計算する。
    """
    _df_valid = pd.DataFrame.from_dict({"time":valid_set.params['time'],
                                       "x_t_i": (2 * preds - 1) * valid_set.params['universe'] * valid_set.params['return']})
    x_t = _df_valid.groupby("time").agg({"x_t_i":"sum"}).values.flatten()
    #embed()
    _mean = x_t.mean()
    _std = x_t.std()
    return ('sigma_score', np.float32(_mean/_std), True)

def build_models(model_params, train_data, validation_data):
    """
    トレーニングデータとバリデーションデータを受け取り、モデルを構築する。
    """
    models = []
    #global config_model
    for _param in model_params:
        _evals_result = {}
        _model = lgb.train(_param, train_data,
                          num_boost_round=200,
                          valid_sets=validation_data,
                          early_stopping_rounds=10,
                          feval=eval_sigma_score,
                          evals_result=_evals_result)
        # 最適ブースティング情報を得る
        _best_boosting_round = _model.best_iteration
        _best_sigma_score = np.max(_evals_result['valid_0']['sigma_score'])
        models.append([_model, _best_boosting_round, _best_sigma_score, _evals_result])
    return models

In [ ]:
while pos_training_start + training_term_range < parse(config_model['validation_term'][0]):
    # 探索処理
    _training_term = [pos_training_start.date(), (pos_training_start + training_term_range).date()]
    print("[Training Term] {} - {}".format(_training_term[0], _training_term[1]))
    
    _idx_training = market_train_df[(market_train_df['time'] >= _training_term[0])
                                    & (market_train_df['time'] <= _training_term[1])].index
    _X_train, _up_train = get_training_data(market_train_df, _idx_training, feature_cols)
    _train_data = lgb.Dataset(_X_train, label=_up_train.astype(int), free_raw_data=False)
    performance_records.append({"training_term":_training_term,
                                "results": build_models(model_params=config_model['model_params'], train_data=_train_data, validation_data=validation_data)})
    
    # サマリを表示
    _latest = performance_records[-1]
    print("=" * 80)
    print("[Training term] {} - {}".format(_latest['training_term'][0], _latest['training_term'][1]))
    for _r in _latest['results']:
        print("Best Sigma Score: {} / boost_rounds = {}".format(_r[2], _r[1]))
    print("=" * 80)
    
    pos_training_start = pos_training_start + training_term_step
    

In [ ]:
ranking_records = sorted(performance_records, key=lambda x:x['results'][0][2], reverse=True)
for _r in ranking_records:
    print("=" * 80)
    print("[Training term] {} - {}".format(_r['training_term'][0], _r['training_term'][1]))
    for _i in _r['results']:
        print("Best Sigma Score: {} / boost_rounds = {}".format(_i[2], _i[1]))
    print("=" * 80)


# アンサンブルモデル構築
アンサンブルを構成する各モデルの設定を定義する。  
各アンサンブルモデルを構築したら、ポストプロセスのパラメータもチューニングする。  
その上で、最後にアンサンブルをして、さらにポストプロセスのパラメータもチューニングする。

In [ ]:
# アンサンブルモデル定義
ensemble_model_config = [
{"training_term":['2012-01-15', '2013-01-15'],
 "boost_rounds":[9,15]
 #Best Sigma Score: 0.7019810080528259 / boost_rounds = 9
 #Best Sigma Score: 0.7038402557373047 / boost_rounds = 15
},

{"training_term":['2012-01-29', '2013-01-29'],
 "boost_rounds":[37,6]
 #Best Sigma Score: 0.7007758021354675 / boost_rounds = 37
 #Best Sigma Score: 0.6739314198493958 / boost_rounds = 6
},

{"training_term":['2012-01-01', '2013-01-01'],
 "boost_rounds": [30,51]
 #Best Sigma Score: 0.6985230445861816 / boost_rounds = 30
 #Best Sigma Score: 0.7033873200416565 / boost_rounds = 51
},

{"training_term":['2012-02-26', '2013-02-26'],
 "boost_rounds":[8,6]
 #Best Sigma Score: 0.6970338225364685 / boost_rounds = 8
 #Best Sigma Score: 0.6925708055496216 / boost_rounds = 6
},

{"training_term":['2012-04-08', '2013-04-08'],
 "boost_rounds": [11,6]
 #Best Sigma Score: 0.6931887865066528 / boost_rounds = 11
 #Best Sigma Score: 0.6818544268608093 / boost_rounds = 6
},

{"training_term":['2012-03-25', '2013-03-25'],
 "boost_rounds":[10,11]
 # Best Sigma Score: 0.6783953309059143 / boost_rounds = 10
 # Best Sigma Score: 0.6836061477661133 / boost_rounds = 11
},

{"training_term": ['2012-03-11', '2013-03-11'],
 "boost_rounds": [9,9]
 #Best Sigma Score: 0.6657338738441467 / boost_rounds = 9
 #Best Sigma Score: 0.6603593230247498 / boost_rounds = 9
},

{"training_term": ['2012-04-22', '2013-04-22'],
 "boost_rounds": [11,18]
 #Best Sigma Score: 0.6643645167350769 / boost_rounds = 11
 #Best Sigma Score: 0.6672964096069336 / boost_rounds = 18
},

{"training_term": ['2015-09-20', '2016-09-20'],
 "boost_rounds": [34,17]
 #Best Sigma Score: 0.6615551710128784 / boost_rounds = 34
 #Best Sigma Score: 0.6718968152999878 / boost_rounds = 17
},

{"training_term": ['2012-05-06', '2013-05-06'],
 "boost_rounds": [17,35]
 #Best Sigma Score: 0.6613066792488098 / boost_rounds = 17
 #Best Sigma Score: 0.6757410764694214 / boost_rounds = 35
},

{"training_term": ['2012-12-16', '2013-12-16'],
 "boost_rounds":[8,11]
 #Best Sigma Score: 0.6613001823425293 / boost_rounds = 8
 #Best Sigma Score: 0.6535777449607849 / boost_rounds = 11
},

{"training_term": ['2012-05-20', '2013-05-20'],
 "boost_rounds" : [11,11]
 #Best Sigma Score: 0.6609295606613159 / boost_rounds = 11
 #Best Sigma Score: 0.6650127172470093 / boost_rounds = 11
},

{"training_term": ['2012-06-03', '2013-06-03'],
 "boost_rounds": [24, 17]
 #Best Sigma Score: 0.6582465171813965 / boost_rounds = 24
 #Best Sigma Score: 0.6538072824478149 / boost_rounds = 17
},

{"training_term": ['2012-12-30', '2013-12-30'],
 "boost_rounds":[5,5]
 #Best Sigma Score: 0.6574293971061707 / boost_rounds = 5
 #Best Sigma Score: 0.654277503490448 / boost_rounds = 5
},

{"training_term": ['2012-07-01', '2013-07-01'],
 "boost_rounds":[19,14]
 #Best Sigma Score: 0.6573091149330139 / boost_rounds = 19
 #Best Sigma Score: 0.6588096022605896 / boost_rounds = 14
},

{"training_term" : ['2012-02-12', '2013-02-12'],
 "boost_rounds":[14,13]
 #Best Sigma Score: 0.6557199954986572 / boost_rounds = 14
 #Best Sigma Score: 0.666266918182373 / boost_rounds = 13
},

{"training_term": ['2012-06-17', '2013-06-17'],
 "boost_rounds":[10,12]
 #Best Sigma Score: 0.6555286645889282 / boost_rounds = 10
 #Best Sigma Score: 0.6548465490341187 / boost_rounds = 12
}
]

In [ ]:
def build_models_without_validation(model_params, train_data, boost_rounds):
    """
    トレーニングデータとブースト回数を指定してモデルを構築する。
    """
    models = []
    #global config_model
    for _param, _boost_round in zip(model_params, boost_rounds):
        print("building model...")
        # 注意！ ブースティング回数はデフォルト状態では1000回になっている！
        _param['num_iteration'] = _boost_round
        _model = lgb.train(_param, train_data,
                          num_boost_round=_boost_round)
        models.append(_model)
        print("model was built.")
    return models

In [ ]:
# モデルを格納する
model_house = []
for _config in ensemble_model_config:
    _idx_training = market_train_df[(market_train_df['time'] >= parse(_config['training_term'][0]).date())
                                    & (market_train_df['time'] <= parse(_config['training_term'][1]).date())].index
    _X_train, _up_train = get_training_data(market_train_df, _idx_training, feature_cols)
    _train_data = lgb.Dataset(_X_train, label=_up_train.astype(int), free_raw_data=False)
    _models = build_models_without_validation(model_params=config_model['model_params'], 
                                              train_data=_train_data,
                                              boost_rounds=_config['boost_rounds'])
    for _each_model in _models:
        model_house.append({"model":_each_model, "post_process_config":{}})

# アンサンブルモデルの検証

In [36]:
preds = []
for _each_model in model_house:
    print("predicting ...")
    preds.append(_each_model['model'].predict(X_valid))


predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...
predicting ...


In [48]:
# 上位スコアのものに限定すると...
total_preds = None
selected_preds = preds[0:5]
for _preds in selected_preds:
    if total_preds is None:
        total_preds = _preds.copy()
    else:
        total_preds += _preds

# 確信度
confidence = 2 * (total_preds/len(selected_preds)) - 1

In [49]:
confidence

array([-0.0574971 , -0.08385243,  0.02016097, ..., -0.03674141,
        0.12124093,  0.21156686])

In [50]:
r_test = validation_data.params['return']
x_t_i = confidence * validation_data.params['return'] * validation_data.params['universe']
data = {'day' : validation_data.params['time'], 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
x_t = df.groupby('day').sum().values.flatten()
mean = np.mean(x_t)
std = np.std(x_t)
score_test = mean / std
print(score_test)

0.7057063598432859


# sigmoid 処理による性能改善
## アンサンブル結果全体に対するSigmoid処理の調整結果
こんな感じ。
```
[post-process][sigmoid:400] sig_score=0.7250044728795221
[post-process][sigmoid:401] sig_score=0.7250023385705368
[post-process][sigmoid:402] sig_score=0.7250001930146031

```

## 上位10位だけに絞ると
素の状態で *0.70747* とかになるが、sigmoid処理をしてもブーストしない。

## 上位20位
```
[post-process][sigmoid:131] sig_score=0.7273554168453827
```

In [37]:
preds[0]

array([0.50327757, 0.48918938, 0.50527056, ..., 0.51525407, 0.5245747 ,
       0.55407184])

In [ ]:
for _sig_coeff in range(1000,10000,500):
    confidence_sig = (2/(1 + np.exp(-1*_sig_coeff * confidence))-1.0)
    x_t_i = confidence_sig * validation_data.params['return'] * validation_data.params['universe']
    data = {'day' : validation_data.params['time'], 'x_t_i' : x_t_i}
    df = pd.DataFrame(data)
    x_t = df.groupby('day').sum().values.flatten()
    mean = np.mean(x_t)
    std = np.std(x_t)
    score_test = mean / std
    print("[post-process][sigmoid:{}] sig_score={}".format(_sig_coeff, score_test))


In [38]:
# 各アンサンブルモデルに対する調整
for _pos in range(len(preds)):
    print("="*80)
    _each_preds = preds[_pos]
    _confidence = _each_preds * 2 - 1.0
    for _sig_coeff in range(1,1000):
        confidence_sig = (2/(1 + np.exp(-1*_sig_coeff * _confidence))-1.0)
        x_t_i = confidence_sig * validation_data.params['return'] * validation_data.params['universe']
        data = {'day' : validation_data.params['time'], 'x_t_i' : x_t_i}
        df = pd.DataFrame(data)
        x_t = df.groupby('day').sum().values.flatten()
        mean = np.mean(x_t)
        std = np.std(x_t)
        score_test = mean / std
        print("[post-process][sigmoid:{}] sig_score={}".format(_sig_coeff, score_test))


[post-process][sigmoid:1] sig_score=0.7020700732734485
[post-process][sigmoid:2] sig_score=0.7023327188104237
[post-process][sigmoid:3] sig_score=0.7027559004195998
[post-process][sigmoid:4] sig_score=0.703319743893958
[post-process][sigmoid:5] sig_score=0.703999854614348
[post-process][sigmoid:6] sig_score=0.704769778539991
[post-process][sigmoid:7] sig_score=0.7056031970378857
[post-process][sigmoid:8] sig_score=0.7064756060089374
[post-process][sigmoid:9] sig_score=0.7073654032396877
[post-process][sigmoid:10] sig_score=0.7082544340293557
[post-process][sigmoid:11] sig_score=0.7091281105171173
[post-process][sigmoid:12] sig_score=0.7099752364176408
[post-process][sigmoid:13] sig_score=0.710787655573127
[post-process][sigmoid:14] sig_score=0.7115598167773362
[post-process][sigmoid:15] sig_score=0.7122883196619079
[post-process][sigmoid:16] sig_score=0.7129714825482626
[post-process][sigmoid:17] sig_score=0.7136089548997948
[post-process][sigmoid:18] sig_score=0.7142013842283021
[post

[post-process][sigmoid:146] sig_score=0.7208654028801557
[post-process][sigmoid:147] sig_score=0.7208614667060804
[post-process][sigmoid:148] sig_score=0.7208574684286276
[post-process][sigmoid:149] sig_score=0.7208534093329083
[post-process][sigmoid:150] sig_score=0.7208492907148778
[post-process][sigmoid:151] sig_score=0.7208451138801647
[post-process][sigmoid:152] sig_score=0.7208408801429091
[post-process][sigmoid:153] sig_score=0.7208365908246144
[post-process][sigmoid:154] sig_score=0.7208322472530152
[post-process][sigmoid:155] sig_score=0.7208278507609646
[post-process][sigmoid:156] sig_score=0.7208234026853408
[post-process][sigmoid:157] sig_score=0.7208189043659801
[post-process][sigmoid:158] sig_score=0.7208143571446325
[post-process][sigmoid:159] sig_score=0.720809762363947
[post-process][sigmoid:160] sig_score=0.7208051213664813
[post-process][sigmoid:161] sig_score=0.720800435493746
[post-process][sigmoid:162] sig_score=0.7207957060852737
[post-process][sigmoid:163] sig_s

[post-process][sigmoid:290] sig_score=0.7201701091904773
[post-process][sigmoid:291] sig_score=0.7201665078436084
[post-process][sigmoid:292] sig_score=0.7201629340928792
[post-process][sigmoid:293] sig_score=0.7201593879003936
[post-process][sigmoid:294] sig_score=0.7201558692250785
[post-process][sigmoid:295] sig_score=0.7201523780227734
[post-process][sigmoid:296] sig_score=0.7201489142463203
[post-process][sigmoid:297] sig_score=0.7201454778456491
[post-process][sigmoid:298] sig_score=0.7201420687678644
[post-process][sigmoid:299] sig_score=0.7201386869573279
[post-process][sigmoid:300] sig_score=0.7201353323557408
[post-process][sigmoid:301] sig_score=0.7201320049022247
[post-process][sigmoid:302] sig_score=0.7201287045333996
[post-process][sigmoid:303] sig_score=0.7201254311834614
[post-process][sigmoid:304] sig_score=0.7201221847842573
[post-process][sigmoid:305] sig_score=0.7201189652653609
[post-process][sigmoid:306] sig_score=0.7201157725541434
[post-process][sigmoid:307] sig

[post-process][sigmoid:435] sig_score=0.7198906401069292
[post-process][sigmoid:436] sig_score=0.7198900298919575
[post-process][sigmoid:437] sig_score=0.7198894326685684
[post-process][sigmoid:438] sig_score=0.7198888483486137
[post-process][sigmoid:439] sig_score=0.7198882768444329
[post-process][sigmoid:440] sig_score=0.7198877180688518
[post-process][sigmoid:441] sig_score=0.7198871719351821
[post-process][sigmoid:442] sig_score=0.7198866383572207
[post-process][sigmoid:443] sig_score=0.7198861172492483
[post-process][sigmoid:444] sig_score=0.7198856085260279
[post-process][sigmoid:445] sig_score=0.7198851121028037
[post-process][sigmoid:446] sig_score=0.7198846278953009
[post-process][sigmoid:447] sig_score=0.719884155819724
[post-process][sigmoid:448] sig_score=0.7198836957927535
[post-process][sigmoid:449] sig_score=0.7198832477315482
[post-process][sigmoid:450] sig_score=0.7198828115537407
[post-process][sigmoid:451] sig_score=0.7198823871774377
[post-process][sigmoid:452] sig_

KeyboardInterrupt: 

In [39]:
_pos = 30
print("="*80)
_each_preds = preds[_pos]
_confidence = _each_preds * 2 - 1.0
for _sig_coeff in range(1,1000):
    confidence_sig = (2/(1 + np.exp(-1*_sig_coeff * _confidence))-1.0)
    x_t_i = confidence_sig * validation_data.params['return'] * validation_data.params['universe']
    data = {'day' : validation_data.params['time'], 'x_t_i' : x_t_i}
    df = pd.DataFrame(data)
    x_t = df.groupby('day').sum().values.flatten()
    mean = np.mean(x_t)
    std = np.std(x_t)
    score_test = mean / std
    print("[post-process][sigmoid:{}] sig_score={}".format(_sig_coeff, score_test))


[post-process][sigmoid:1] sig_score=0.6557174251712714
[post-process][sigmoid:2] sig_score=0.6557089602381052
[post-process][sigmoid:3] sig_score=0.6556928080465628
[post-process][sigmoid:4] sig_score=0.6556672038759144
[post-process][sigmoid:5] sig_score=0.6556314989646533
[post-process][sigmoid:6] sig_score=0.6555867456444957
[post-process][sigmoid:7] sig_score=0.6555356386594939
[post-process][sigmoid:8] sig_score=0.65548199297886
[post-process][sigmoid:9] sig_score=0.6554300540511769
[post-process][sigmoid:10] sig_score=0.6553838795550812
[post-process][sigmoid:11] sig_score=0.6553469120053466
[post-process][sigmoid:12] sig_score=0.65532175980233
[post-process][sigmoid:13] sig_score=0.6553101476994224
[post-process][sigmoid:14] sig_score=0.6553129798757376
[post-process][sigmoid:15] sig_score=0.6553304635260897
[post-process][sigmoid:16] sig_score=0.6553622543360768
[post-process][sigmoid:17] sig_score=0.6554075993142567
[post-process][sigmoid:18] sig_score=0.6554654637651075
[post

[post-process][sigmoid:146] sig_score=0.6590337035971732
[post-process][sigmoid:147] sig_score=0.6590307736551647
[post-process][sigmoid:148] sig_score=0.6590278927216136
[post-process][sigmoid:149] sig_score=0.6590250609726529
[post-process][sigmoid:150] sig_score=0.6590222785446149
[post-process][sigmoid:151] sig_score=0.6590195455369997
[post-process][sigmoid:152] sig_score=0.6590168620152422
[post-process][sigmoid:153] sig_score=0.6590142280132996
[post-process][sigmoid:154] sig_score=0.6590116435360592
[post-process][sigmoid:155] sig_score=0.659009108561587
[post-process][sigmoid:156] sig_score=0.6590066230432224
[post-process][sigmoid:157] sig_score=0.659004186911527
[post-process][sigmoid:158] sig_score=0.6590018000761029


KeyboardInterrupt: 

In [ ]:
#1個目のデータと2個目のデータの平均出してる
confidence_test = (gbm_1.predict(X_test) + gbm_2.predict(X_test))/2
confidence_test

In [ ]:
#予測値の%　割合の算出
confidence_test = (confidence_test-confidence_test.min())/(confidence_test.max()-confidence_test.min())
confidence_test

In [ ]:
#%に二乗して-1????
confidence_test = confidence_test*2-1
print(max(confidence_test),min(confidence_test))
confidence_test

In [ ]:
# 最終スコアの計算に使用される実際のメトリックの計算
r_test = r_test.clip(-1,1) # -1～１以外の値を取り除く　彼らはどこから来たのかという
#学習の推測地と予測前の目的変数とユニバース値をかけてる？
x_t_i = confidence_test * r_test * u_test
#日付とスコア値だけのデータフレーム作り
data = {'day' : d_test, 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
#日付でグループバイ（集約）　で多次元配列を１次元に直してる
x_t = df.groupby('day').sum().values.flatten()
#スコアの平均値
mean = np.mean(x_t)
#スコアの標準偏差
std = np.std(x_t)
#変動係数の逆数
score_test = mean / std
print(score_test)

In [ ]:
import gc
del X_train,X_test
gc.collect()

In [ ]:
#prediction
days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
total_market_obs_df = []
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if (n_days%50==0):
        print(n_days,end=' ')
    t = time.time()
    market_obs_df['time'] = market_obs_df['time'].dt.date
    
    return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
    total_market_obs_df.append(market_obs_df)
    if len(total_market_obs_df)==1:
        history_df = total_market_obs_df[0]
    else:
        history_df = pd.concat(total_market_obs_df[-(np.max(n_lag)+1):])
    print(history_df)
    
    new_df = generate_lag_features(history_df,n_lag=[3,7,14])
    market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
#     return_features = ['open']
#     new_df = generate_lag_features(market_obs_df,n_lag=[3,7,14])
#     market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
    market_obs_df = mis_impute(market_obs_df)
    
    market_obs_df = data_prep(market_obs_df)
    
#     market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = (gbm_1.predict(X_live) + gbm_2.predict(X_live))/2
    prediction_time += time.time() -t
    
    t = time.time()
    
    confidence = lp
    confidence = (confidence-confidence.min())/(confidence.max()-confidence.min())
    confidence = confidence * 2 - 1
    
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t
    
env.write_submission_file()
sub  = pd.read_csv("submission.csv")